In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
L = np.array([
    [0,    0,    0,    0,    2.833, 3.667, 4.000, 6.833, 6.166, 2.500, 0],
    [0.825, 0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
    [0,    0.697, 0,    0,    0,    0,    0,    0,    0,    0,    0],
    [0,    0,    0.652, 0,    0,    0,    0,    0,    0,    0,    0],
    [0,    0,    0,    0.533, 0,    0,    0,    0,    0,    0,    0],
    [0,    0,    0,    0,    0.750, 0,    0,    0,    0,    0,    0],
    [0,    0,    0,    0,    0,    1.000, 0,    0,    0,    0,    0],
    [0,    0,    0,    0,    0,    0,    1.000, 0,    0,    0,    0],
    [0,    0,    0,    0,    0,    0,    0,    1.000, 0,    0,    0],
    [0,    0,    0,    0,    0,    0,    0,    0,    1.000, 0,    0],
    [0,    0,    0,    0,    0,    0,    0,    0,    0,    0.667, 0]
])


In [3]:
eigvals, eigvecs = np.linalg.eig(L)

# Filtrar apenas os autovalores reais
real_eigvals = eigvals[np.isreal(eigvals)]

# Encontrar o autovalor real dominante (de maior módulo)
dominant_real_eigval = real_eigvals[np.argmax(np.abs(real_eigvals))]

# Encontrar o índice do autovalor dominante
dominant_index = np.argmax(np.abs(real_eigvals))

# Obter o autovetor correspondente ao autovalor dominante
dominant_real_eigvec = eigvecs[:, dominant_index]
dominant_real_eigvec = np.real(dominant_real_eigvec/ dominant_real_eigvec[0])

In [4]:
#Calculando h na colheita uniforme
h = np.real(1 - (1/dominant_real_eigval))
print(f"Taxa de colheita uniforme: h = {h*100:.2f}%")
print('\n Autovalor:')
print(pd.DataFrame(dominant_real_eigvec))

Taxa de colheita uniforme: h = 17.31%

 Autovalor:
           0
0   1.000000
1   0.682201
2   0.393191
3   0.211987
4   0.093432
5   0.057945
6   0.047915
7   0.039622
8   0.032763
9   0.027092
10  0.014943


In [5]:
# Cálculo de h para a colheita da faixa etária mais jovem

I = np.eye(L.shape[0])
# Calculando o determinante de (L-I)
R = np.linalg.det(L-I) + 1 
h_jovem = 1 - (1/R)
print(f"Taxa de colheita para os mais jovens: h = {h_jovem*100:.2f}%")

H = matrix = np.zeros((11, 11))
matrix[0, 0] = h_jovem

IHL = (I - H) @ L

#autovalores e autovetores para a matriz (I-H)L
eigvals, eigvecs = np.linalg.eig(IHL)

# Filtrar apenas os autovalores reais
real_eigvals = eigvals[np.isreal(eigvals)]

# Encontrar o autovalor real dominante (de maior módulo)
dominant_real_eigval = real_eigvals[np.argmax(np.abs(real_eigvals))]
# Encontrar o índice do autovalor dominante
dominant_index = np.argmax(np.abs(real_eigvals))

# Obter o autovetor correspondente ao autovalor dominante
dominant_real_eigvec = np.real(eigvecs[:, dominant_index])
x1 = dominant_real_eigvec/dominant_real_eigvec[0]
print('\n Autovalor:')
print(pd.DataFrame(x1))

Lx1 = L@x1
total_entries_Lx1 = sum(Lx1)
first_entry_percentage = Lx1[0]/total_entries_Lx1
total_collect = first_entry_percentage * h_jovem
print(f"\nTaxa de colheita para toda a população = {total_collect*100:.2f}%")


Taxa de colheita para os mais jovens: h = 75.24%

 Autovalor:
           0
0   1.000000
1   0.825000
2   0.575025
3   0.374916
4   0.199830
5   0.149873
6   0.149873
7   0.149873
8   0.149873
9   0.149873
10  0.099965

Taxa de colheita para toda a população = 44.27%


In [6]:
from scipy.optimize import minimize

#Maximizando a colheita

m = L.shape[0]
initial_survival = np.ones(m)  # (I-H).Chute inicial para as proporções de sobrevivência

# Construção da matriz (I-H)L
def build_matrix(L, O):
    D = np.diag(O)
    return np.dot(L, D)

# Computando o u_i (proporção do i-ésimo grupo etário em relação a população total)
def stable_state(C):
    eigvals, eigvecs = np.linalg.eig(C)
    idx = np.argmin(np.abs(eigvals - 1))  # Encontrar o lambda mais próximo de 1
    y = np.real(eigvecs[:, idx])
    return y / np.sum(y)  #u_i

# O objectivo da função é maximizar G
def harvest_function(O):
    C = build_matrix(L, O)
    u = stable_state(C)
    h = 1 - O  # Taxa de colheita
    G = np.dot(h, u) 
    return -G  # Negativo para ser maximizado na função minimize

# Restrições
def eigenvalue_constraint(O):
    C = build_matrix(L, O)
    eigvals = np.linalg.eigvals(C)
    dominant = max(eigvals.real)
    return 1 - dominant  #autovalor = 1

constraints = [
    {"type": "eq", "fun": eigenvalue_constraint},  # lambda = 1 
    {"type": "ineq", "fun": lambda O: O},          # O >= 0
]

# Limites para a matriz O que é a (I-H): 0 <= O_i <= 1
bounds = [(0, 1)] * m

# Problema de otimização
result = minimize(harvest_function, initial_survival, constraints=constraints, bounds=bounds, method='SLSQP')

# Extract results
optimal_O = result.x # taxa de sobrevivência ótima
optimal_G = -result.fun  # Colheita global máxima

# Print results
print("Taxa de sobrevivência ótima (O):", pd.DataFrame(optimal_O),"\n")
print("Taxas de colheita (h):", pd.DataFrame(1 - optimal_O),"\n")
print(f"Colheita global máxima: {optimal_G*100:.2f}%", )

Taxa de sobrevivência ótima (O):                0
0   2.476429e-01
1   1.000000e+00
2   1.000000e+00
3   1.000000e+00
4   1.000000e+00
5   1.000000e+00
6   1.000000e+00
7   1.000000e+00
8   1.000000e+00
9   1.000000e+00
10  4.440892e-16 

Taxas de colheita (h):                0
0   7.523571e-01
1   0.000000e+00
2   2.220446e-16
3   0.000000e+00
4   0.000000e+00
5   1.110223e-16
6   2.220446e-16
7   0.000000e+00
8   0.000000e+00
9   0.000000e+00
10  1.000000e+00 

Colheita global máxima: 45.73%
